<a href="https://colab.research.google.com/github/Last-Vega/Klis_Workshop_MachineLearning/blob/master/ML6.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# torchtextというパッケージをアップデートしておく（古いバージョンの場合は後で出現する単語埋め込みが読み込めない）
!pip install -U torchtext

     |████████████████████████████████| 4.5MB 4.7MB/s 
     |████████████████████████████████| 1.1MB 22.9MB/s 
  Found existing installation: torchtext 0.3.1
    Uninstalling torchtext-0.3.1:
      Successfully uninstalled torchtext-0.3.1


In [ ]:
# 分かち書きを行うためにjanomeというPythonパッケージをインストール
!pip install janome

     |████████████████████████████████| 19.7MB 1.3MB/s 


In [ ]:
# ファイルのダウンロード
!wget "https://drive.google.com/uc?export=download&id=1c0tXuRt2GE8szurDI01P0YkbgfEuNi6o" -O twitterJSA_data.csv

--2020-10-25 17:24:42--  https://drive.google.com/uc?export=download&id=1c0tXuRt2GE8szurDI01P0YkbgfEuNi6o
Resolving drive.google.com (drive.google.com)... 74.125.20.102, 74.125.20.113, 74.125.20.138, ...
Connecting to drive.google.com (drive.google.com)|74.125.20.102|:443... connected.
HTTP request sent, awaiting response... 302 Moved Temporarily
Location: https://doc-10-58-docs.googleusercontent.com/docs/securesc/ha0ro937gcuc7l7deffksulhg5h7mbp1/om6div6l7ohuk44uje0a4s6bessofoe7/1603646625000/07803272131756145988/*/1c0tXuRt2GE8szurDI01P0YkbgfEuNi6o?e=download [following]
--2020-10-25 17:24:42--  https://doc-10-58-docs.googleusercontent.com/docs/securesc/ha0ro937gcuc7l7deffksulhg5h7mbp1/om6div6l7ohuk44uje0a4s6bessofoe7/1603646625000/07803272131756145988/*/1c0tXuRt2GE8szurDI01P0YkbgfEuNi6o?e=download
Resolving doc-10-58-docs.googleusercontent.com (doc-10-58-docs.googleusercontent.com)... 74.125.195.132, 2607:f8b0:400e:c09::84
Connecting to doc-10-58-docs.googleusercontent.com (doc-10-58-

In [ ]:
from torchtext import data
from janome.tokenizer import Tokenizer # janomeからTokenizerクラスをimportする

# janomeはPythonで書かれた形態素解析器であり単語の分かち書きなどが可能
tokenizer = Tokenizer(wakati=True)

def tokenize(text):
    # 単語の分かち書きを行う
    return list(tokenizer.tokenize(text))

# Fieldオブジェクトは読み込んだデータの各項目（CSVの場合は各列）をどのように処理するかを決定する
# Fieldの引数に何も指定しない場合には，テキストのような系列データと仮定し指定されたtokenizerで処理される
# 今回用いるファイル中のidやlabelのように，系列データでなく，tokenizeする必要もないデータの場合は
# sequential=False, use_vocab=Falseを指定する
ID = data.Field(sequential=False, use_vocab=False)
LABEL = data.Field(sequential=False, use_vocab=False)
TEXT = data.Field(tokenize=tokenize) # tokenize引数に指定した関数でテキストを処理する

# CSVファイルを読み込み
dataset = data.TabularDataset(
    path='./twitterJSA_data.csv', # 読み込みファイル
    format='csv', # 読み込むファイルの形式
    fields=[('id', ID), ('label', LABEL), ('text', TEXT)], # 各列ごとにFieldオブジェクトを設定
    skip_header=True # 最初の行は各列の見出しなので読み込まない
    )

/usr/local/lib/python3.6/dist-packages/torchtext/data/field.py:150: UserWarning: Field class will be retired in the 0.8.0 release and moved to torchtext.legacy. Please see 0.7.0 release notes for further information.
  warnings.warn('{} class will be retired in the 0.8.0 release and moved to torchtext.legacy. Please see 0.7.0 release notes for further information.'.format(self.__class__.__name__), UserWarning)
/usr/local/lib/python3.6/dist-packages/torchtext/data/example.py:68: UserWarning: Example class will be retired in the 0.8.0 release and moved to torchtext.legacy. Please see 0.7.0 release notes for further information.
  warnings.warn('Example class will be retired in the 0.8.0 release and moved to torchtext.legacy. Please see 0.7.0 release notes for further information.', UserWarning)
/usr/local/lib/python3.6/dist-packages/torchtext/data/example.py:78: UserWarning: Example class will be retired in the 0.8.0 release and moved to torchtext.legacy. Please see 0.7.0 release notes f

In [ ]:
from torch.utils.data import random_split

train_size = 20000 # 先頭の20,000件を訓練データとして用いることにする
batch_size = 32 # ミニバッチのサイズを設定

# 訓練データとテストデータに分割
train_dataset, test_dataset = dataset.split(split_ratio=train_size/len(dataset))

# 訓練データを読み込むためのイテレータを準備
train_iterator = data.BucketIterator(train_dataset, batch_size=batch_size, train=True)
test_iterator = data.BucketIterator(test_dataset, batch_size=batch_size, train=False, sort=False)


/usr/local/lib/python3.6/dist-packages/torchtext/data/iterator.py:48: UserWarning: BucketIterator class will be retired in the 0.8.0 release and moved to torchtext.legacy. Please see 0.7.0 release notes for further information.
  warnings.warn('{} class will be retired in the 0.8.0 release and moved to torchtext.legacy. Please see 0.7.0 release notes for further information.'.format(self.__class__.__name__), UserWarning)


In [ ]:
# TEXTフィールド中の単語を収集して，単語の種類数や頻度などを計算し，これをVocabクラスのオブジェクトしてまとめて，
# TEXTフィールドのデータ属性として保存する
TEXT.build_vocab(train_dataset, min_freq=5) # `min_freq=5`: 頻度が5回未満の単語は無視する設定

In [ ]:
# TEXT.vocab.itos # 訓練データ中の全単語のリスト

['<unk>',
 '<pad>',
 'の',
 '。',
 'て',
 'に',
 'が',
 ' ',
 'た',
 '、',
 'し',
 'は',
 'で',
 'ない',
 'だ',
 'な',
 'と',
 'も',
 'を',
 'ん',
 'から',
 '！',
 'けど',
 'Xperia',
 '…',
 'か',
 'iPhone',
 '6',
 'てる',
 'シャープ',
 'なっ',
 '・',
 'よ',
 'たら',
 'する',
 'って',
 'さ',
 '？',
 'ね',
 'Z',
 'コピー',
 'いい',
 '画面',
 '5',
 '機',
 'う',
 'とか',
 'です',
 '  ',
 'コンビニ',
 '(',
 'なく',
 's',
 'セブン',
 'プリント',
 '-',
 '3',
 'XPERIA',
 ')',
 'SH',
 'ある',
 'れ',
 'のに',
 'や',
 'AQUOS',
 'スマホ',
 'こと',
 'ので',
 'なる',
 '使っ',
 '電池',
 'でき',
 'すぎ',
 'ー',
 'これ',
 'い',
 'だっ',
 'き',
 'わ',
 '1',
 '印刷',
 'だけ',
 '機種',
 'じゃ',
 '今',
 '使い',
 'バッテリー',
 'なぁ',
 '2',
 '4',
 'まで',
 'そう',
 'なら',
 'いる',
 'カメラ',
 '思っ',
 '「',
 '充電',
 '」',
 'この',
 'さん',
 'ば',
 'よう',
 'ルンバ',
 'ます',
 'より',
 '#',
 'だろ',
 'できる',
 'もう',
 '前',
 'なかっ',
 'まし',
 '何',
 '（',
 '\u3000',
 '年',
 'でも',
 '綺麗',
 '良い',
 'SHARP',
 '）',
 '写真',
 'ω',
 '経営',
 '携帯',
 '便利',
 '持ち',
 '気',
 'たい',
 '買っ',
 '機能',
 '的',
 '電源',
 'という',
 '方',
 'しか',
 '変え',
 '円',
 'パナソニック',
 '見',
 'み',
 '時',
 'アプリ',
 '出'

In [ ]:
for batch in train_iterator:
    # `batch_size`ごとにデータを取り出す
    print("textのサイズ", batch.text.shape) # textという属性の行数，列数を確認
    print("textの中身", batch.text) # textという属性の内容を確認
    single_text = batch.text[:, 0] # 1個目のテキスト（batch.text[0]ではないことに注意．1列1列が1つのテキスト．`[:, 0]` は1列目を取り出す操作）
    print("1つ目のtext", single_text) 

    # 1つ目のtextの各数字iはTEXT.vocab.itosのi番目の単語に対応しているため，
    # textの各数字を使って元の内容を復元してみる．
    for idx in single_text:
        print(TEXT.vocab.itos[idx])
    break

textのサイズ torch.Size([80, 32])
textの中身 tensor([[  29,  574,   29,  ...,   23,   43,   64],
        [   2, 1087,  124,  ..., 4666,  187,    2],
        [ 374,  284,  171,  ..., 5615,   20,   65],
        ...,
        [   1,    1,    1,  ...,    1,    1,    1],
        [   1,    1,    1,  ...,    1,    1,    1],
        [   1,    1,    1,  ...,    1,    1,    1]])
1つ目のtext tensor([  29,    2,  374,   18,  198,  321,    6,    0,   10,    8,    3, 1110,
           9,   29,    2,    0, 1956,   17, 2298,    5, 4776,  375,  640,  400,
         138,    2,   66,   18,    0,   61,    8,    2,  107,   45,    3,  783,
          63,    0,   16,  250,    0,   14,    3, 3583,   34,  263,    5,   11,
           0,  676,    6,   60,    3,   96,    0,   98,   16,  366,    3,    1,
           1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
           1,    1,    1,    1,    1,    1,    1,    1])
シャープ
の
買収
を
ホン
ハイ
が
<unk>
し
た
。
多分
、
シャープ
の
<unk>
長
も
直前
に
危ない
債務
3500
億
円
の
こと
を
<unk>
れ


/usr/local/lib/python3.6/dist-packages/torchtext/data/batch.py:23: UserWarning: Batch class will be retired in the 0.8.0 release and moved to torchtext.legacy. Please see 0.7.0 release notes for further information.
  warnings.warn('{} class will be retired in the 0.8.0 release and moved to torchtext.legacy. Please see 0.7.0 release notes for further information.'.format(self.__class__.__name__), UserWarning)


In [ ]:
from torchtext.vocab import FastText

# FastTextの単語埋め込みを設定し，TEXT.vocabを更新
# 単語埋め込みをダウンロードする必要があるため少し時間がかかる．
TEXT.build_vocab(train_dataset, min_freq=5, vectors=FastText(language="ja"))

.vector_cache/wiki.ja.vec: 1.37GB [00:25, 53.2MB/s]                            
100%|█████████▉| 579471/580000 [01:02<00:00, 9436.57it/s]

In [ ]:
# 全単語中で2000番目の単語は「応援」，この単語の埋め込みを表示
print(TEXT.vocab.itos[2000])
print(TEXT.vocab.vectors[2])

涙
tensor([  2.4798,  -2.3370,  -2.4400, -12.1400,  -0.8372,  -1.9392,   2.8368,
          1.6653,   5.0867,  -4.0020,   0.3909,   4.1181,  -2.5103,  -3.3195,
          3.3858,  -1.0409,  -7.0832,  -3.5734,  -4.2110,   3.2943,   2.3502,
         -4.3066,  -2.6962,  -1.1227,   2.2315,  -6.9947,   4.2679,   2.0828,
          2.7985,  -6.0795,  -0.2957,  -1.8515,  -6.9696,  -8.5109,  -5.2566,
         -4.0461,  -4.1703,  -8.5942,   7.4083,  -4.8971,   2.7055,   4.1092,
          2.9140,  -4.2971,  -4.3403,  -2.2493,   1.8756,   3.3291,   2.8136,
          1.9454,  -4.9651,   3.3335,  -1.0476,   7.1598,   2.1137,  -1.3449,
         -1.3332,   4.1351,  -4.1717,   3.1894,   0.5255,   2.5963,  -0.2207,
          4.1619,  -3.5947,  -1.2783,   4.8074,   4.7244,   9.8638,  -6.9120,
          3.7264,   4.2430,  -6.3267,   5.9991,  -6.2397,   0.4630,   0.8871,
          3.3392,  -4.6438,   5.2737,  -2.8144,   7.4055,   2.6909,  17.4130,
          0.7899,   0.7587,   3.0321,   5.3304,  -2.9807,  -4.

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class TextClassifier(nn.Module):
    def __init__(self, vocab):
        super(TextClassifier, self).__init__()

        # 単語埋め込み（FastText）を設定する
        self.embedding = torch.nn.Embedding.from_pretrained(
            embeddings=vocab.vectors, freeze=False)
        
        emb_size = vocab.vectors.shape[1]
        self.fc1 = nn.Linear(emb_size, 100) # 100 x 300 の行列を含む全結合層を設定
        self.fc2 = nn.Linear(100, 1) # 1 x 100 の行列（この場合はベクトル）を含む全結合層を設定

    def forward(self, seq):
        x = self.embedding(seq) # 各単語を単語埋め込みに変換する
        x = x.mean(axis=0) # 各テキストに含まれるすべての単語の埋め込みの平均をとる
        # x = F.relu(self.fc1(x))
        x = self.fc1(x)
        h = self.fc2(x)
        h = h.squeeze()
        return h

In [ ]:
import torch
import torch.optim as optim

device = torch.device("cuda" if torch.cuda.is_available() else "cpu") # CPUもしくはGPUのどちらを使うかを設定
model = TextClassifier(TEXT.vocab) # ニューラルネットワークモデルのインスタンスを生成
model = model.to(device) # CPUもしくはGPUのどちらを設定
optimizer = optim.Adam(model.parameters()) # 基本的な学習方法はミニバッチ勾配降下法ではあるが，その中でもよく用いられるAdamと呼ばれる方法を用いることにする

criterion = nn.BCEWithLogitsLoss() # 二値分類用の交差エントロピーを最小化することにする

epoch_size = 10 # 勾配降下法はすべてのデータでパラメータを更新したら終わりではなく，全データでの更新（=1エポック）を複数回行う必要がある

model.train() # モデルを学習モードに変更

# `epoch_size`の数だけ以下を繰り返す
for epoch in range(epoch_size):
    losses = []
    # イテレータはミニバッチ勾配降下法のために，`batch_size`で指定した数ごとにデータをわけて読み込んでくれる．
    for batch_idx, batch in enumerate(train_iterator):
        texts, labels = batch.text, batch.label
        optimizer.zero_grad() # 勾配の初期化
        y = model(texts) # 現時点でのモデルの出力を得る
        loss = criterion(y, labels.type(torch.float)) # 交差エントロピーの計算
        loss.backward() # 交差エントロピーの勾配計算
        optimizer.step() # パラメータ更新
        losses.append(loss.item())

    # 現在の交差エントロピーを出力
    print('Epoch: {}\tCross Entropy: {:.6f}'.format(epoch, sum(losses)))

/usr/local/lib/python3.6/dist-packages/torchtext/data/batch.py:23: UserWarning: Batch class will be retired in the 0.8.0 release and moved to torchtext.legacy. Please see 0.7.0 release notes for further information.
  warnings.warn('{} class will be retired in the 0.8.0 release and moved to torchtext.legacy. Please see 0.7.0 release notes for further information.'.format(self.__class__.__name__), UserWarning)
100%|█████████▉| 579471/580000 [01:20<00:00, 9436.57it/s]

Epoch: 0	Cross Entropy: 420.232098
Epoch: 1	Cross Entropy: 298.417847
Epoch: 2	Cross Entropy: 212.551775
Epoch: 3	Cross Entropy: 179.033084
Epoch: 4	Cross Entropy: 160.672665
Epoch: 5	Cross Entropy: 150.084422
Epoch: 6	Cross Entropy: 135.034220
Epoch: 7	Cross Entropy: 125.557820
Epoch: 8	Cross Entropy: 122.725942
Epoch: 9	Cross Entropy: 121.487459


In [ ]:
correct = 0
model.eval() # モデルを評価モードに変更
for batch_idx, batch in enumerate(test_iterator):
    texts, labels = batch.text, batch.label
    y = model(texts) # モデルの出力を得る
    result = torch.sigmoid(y) # `TextClassifier`ではsigmoid関数を適用していなかったのでここで適用
    prediction = result >= 0.5 # `result`ベクトルと同じ次元を持ち，`result`の中で0.5以上である次元がTrue，それ以外がFalseであるベクトルを`prediction`とする
    target = labels == 1 # `labels`ベクトルと同じ次元を持ち，`labels`の中で1である次元がTrue，それ以外がFalseであるベクトルを`target`とする
    correct_num = target.eq(prediction).sum().item() # `prediction`ベクトルと`target`ベクトルでTrue/Falseが一致したものの数を数える
    correct += correct_num

# test_iterator.datasetにはテストデータ全体が入っているので，これの長さはテストデータの事例数となる
print("Accuracy: {:.3f}".format(correct / len(test_iterator.dataset)))

/usr/local/lib/python3.6/dist-packages/torchtext/data/batch.py:23: UserWarning: Batch class will be retired in the 0.8.0 release and moved to torchtext.legacy. Please see 0.7.0 release notes for further information.
  warnings.warn('{} class will be retired in the 0.8.0 release and moved to torchtext.legacy. Please see 0.7.0 release notes for further information.'.format(self.__class__.__name__), UserWarning)


Accuracy: 0.891
